PyTorchViz examples
==========

In [1]:
import torch
from torch import nn
%pip install -U git+https://github.com/szagoruyko/pytorchviz.git@master
from torchviz import make_dot, make_dot_from_trace

  Cloning https://github.com/szagoruyko/pytorchviz.git (to revision master) to /tmp/pip-req-build-ydf41hkl
  Running command git clone --filter=blob:none --quiet https://github.com/szagoruyko/pytorchviz.git /tmp/pip-req-build-ydf41hkl
  Resolved https://github.com/szagoruyko/pytorchviz.git to commit 0adcd83af8aa7ab36d6afd139cabbd9df598edb7
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4973 sha256=1bbc8fb0fcb3d6f14ddfe8d442f343f886fbe6353b852a2f9037ab572238fd2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-da1j7bku/wheels/97/11/17/d09e895f9883bd50923ce86b249a31790356a1b88b280983d8
Successfully built torchviz


In [10]:
import warnings
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import argparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.preprocessing import LabelEncoder

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, output_size)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.activation(self.input_layer(x))
        x = self.dropout(x)  # Aplicar o dropout após a ativação
        x = self.output_layer(x)
        # Aplicar a função Softmax na camada de saída
        x = nn.functional.softmax(x, dim=1)
        return x

In [23]:
def create_loaders(X_train, X_test, y_train, y_test):
    #print("\n")
    #print("Shape X_train: "+str(X_train.shape))
    #print("Shape X_test: " + str(X_test.shape))
    #print("Shape y_train: " + str(y_train.shape))
    #print("Shape Y_test: " + str(y_test.shape))
    #exit()

    # Crie conjuntos de dados para treinamento e teste
    train_dataset = CustomDataset(X_train, y_train)
    test_dataset = CustomDataset(X_test, y_test)

    # Crie os data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    return train_loader, test_loader

In [24]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_sample = self.X[idx]
        y_sample = self.y[idx]
        return x_sample, y_sample

def remove_spaces(column_name):
    return column_name.strip()

def load_dataset(dataset_id):


    if dataset_id == 1:
        # Caminho para o diretório do conjunto de dados
        data_dir = "../dataset/cic-unb-ids/Tuesday-WorkingHours.pcap_ISCX.csv"
    elif dataset_id == 2:
        # Caminho para o diretório do conjunto de dados
        data_dir = "../dataset/cic-unb-ids/Wednesday-workingHours.pcap_ISCX.csv"
    elif dataset_id == 3:
        # Caminho para o diretório do conjunto de dados
        data_dir = "../dataset/cic-unb-ids/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"
    elif dataset_id == 4:
        # Caminho para o diretório do conjunto de dados
        data_dir = "./Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"
        #data_dir = ""
    elif dataset_id == 5:
        # Caminho para o diretório do conjunto de dados
        data_dir = "../dataset/cic-unb-ids/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"
    elif dataset_id == 6:
        # Caminho para o diretório do conjunto de dados
        #data_dir = "../dataset/cic-unb-ids/Friday-WorkingHours-Morning.pcap_ISCX.csv"
        data_dir = ""
    elif dataset_id == 7:
        # Caminho para o diretório do conjunto de dados
        #data_dir = "../dataset/cic-unb-ids/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
        data_dir = ""


    df = pd.read_csv(data_dir)
    #df = column_remover(df)
    df.columns = df.columns.map(remove_spaces)

    df['Label'] = df['Label'].apply(lambda x: 'MALIGNANT' if x != 'BENIGN' else x)

    # Separar as colunas de recursos (features) e rótulos (labels)
    X = df.drop('Label', axis=1)  # Substitua 'label' pelo nome da coluna de rótulos
    y = df['Label']

    # Dividir os dados em conjuntos de treinamento e teste
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    X_train = X_train.replace([np.inf, -np.inf], np.nan)
    X_test = X_test.replace([np.inf, -np.inf], np.nan)

    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)


    # Crie um objeto LabelEncoder
    label_encoder = LabelEncoder()

    # Ajuste o LabelEncoder aos seus dados de classe (y_train)
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.fit_transform(y_test)

    # Crie um tensor PyTorch a partir dos valores codificados
    y_train = torch.tensor(y_train_encoded, dtype=torch.int64)
    y_test = torch.tensor(y_test_encoded, dtype=torch.int64)


    # Padronizar os recursos (opcional, mas geralmente recomendado)
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    # Converter os dados para tensores PyTorch
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    #y_train = torch.tensor(y_train.values, dtype=torch.int64)
    #y_test = torch.tensor(y_test.values, dtype=torch.int64)

    return X_train, X_test, y_train, y_test

In [25]:
X_train, X_test, y_train, y_test = load_dataset(4)
train_loader, val_loader = create_loaders(X_train, X_test, y_train, y_test)

In [32]:
model_path = './cic_unb_server_model_aggregated.pth'
print(model_path)

model = LSTMModel(input_size=78, hidden_size=16, num_layers=5, output_size=2)
print(model)
model.eval()
model.load_state_dict(torch.load(model_path))


data, targets = next(iter(val_loader))

# Certifique-se de que os dados estão na mesma device que o seu modelo


# Agora você pode passar os dados para o seu modelo
output = model(data)

# E então você pode visualizar o gráfico
dot = make_dot(output, params=dict(model.named_parameters()))
dot.render(filename='output_graph', format='pdf')




./cic_unb_server_model_aggregated.pth
LSTMModel(
  (input_layer): Linear(in_features=78, out_features=16, bias=True)
  (hidden_layer): Linear(in_features=16, out_features=16, bias=True)
  (output_layer): Linear(in_features=16, out_features=2, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
)


'output_graph.pdf'

Specify `show_attrs=True` and `show_saved=True` to see what autograd saves for the backward pass

In [33]:
dot2 = make_dot(output, params=dict(model.named_parameters()), show_attrs=True, show_saved=True)
dot2.render(filename='output_graph_2', format='pdf')

'output_graph_2.pdf'